In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets.cifar10 import load_data
# keras의 helper 함수인 load_data 함수 Import.

In [3]:
# next_batch 유틸리티 함수를 정의
def next_batch(num, data, labels):
  '''
  `num` 개수 만큼의 랜덤한 샘플들과 레이블들을 리턴합니다.
  '''
  idx = np.arange(0 , len(data))
  np.random.shuffle(idx)
  idx = idx[:num]
  data_shuffle = [data[ i] for i in idx]
  labels_shuffle = [labels[ i] for i in idx]

  return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [4]:
# CNN 
def build_CNN_classifier(x):
  x_image = x

  # convolutional layer 1 - 하나의 grayscale 이미지를 64개의 특징들(feature)으로 맵핑(maping)
  W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 3, 64], stddev=5e-2))
  b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)

  # Pooling layer 1
  h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

  # convolutional layer 2 - 32개의 특징들(feature)을 64개의 특징들(feature)로 맵핑(maping)
  W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 64], stddev=5e-2))
  b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

  # pooling layer 2
  h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

  # convolutional layer 3
  W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
  b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)

  # convolutional layer 4
  W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
  b_conv4 = tf.Variable(tf.constant(0.1, shape=[128])) 
  h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)

  # convolutional layer 5
  W_conv5 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
  b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)

  # Fully Connected Layer 1 - 2번의 downsampling 이후에, 우리의 32x32 이미지는 8x8x128 특징맵(feature map)이 됨.
  # 이를 384개의 특징들로 맵핑(maping)
  W_fc1 = tf.Variable(tf.truncated_normal(shape=[8 * 8 * 128, 384], stddev=5e-2))
  b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))

  h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])
  h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

  # Dropout - 모델의 복잡도 컨트롤. 특징들의 co-adaptation 방지
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) 

  # Fully Connected Layer 2 - 384개의 특징들(feature)을 10개의 클래스-airplane, automobile, bird...-로 맵핑(maping)
  W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 10], stddev=5e-2))
  b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
  logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
  y_pred = tf.nn.softmax(logits)

  return y_pred, logits

In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# 인풋 아웃풋 데이터, 드롭아웃 확률을 입력받기위한 플레이스홀더 정의
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
# CIFAR-10 데이터 다운로드 & 불러오기
(x_train, y_train), (x_test, y_test) = load_data()
# scalar 형태의 레이블(0~9)을 One-hot Encoding 형태로 변환
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10),axis=1)

170498071/170498071 [==============================] - 12s 0us/step


In [8]:
# Convolutional Neural Networks(CNN) 그래프 생성
y_pred, logits = build_CNN_classifier(x)

# Cross Entropy를 비용함수(loss function)으로 정의, RMSPropOptimizer를 이용해서 비용 함수를 최소화
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
# 정확도 계산 연산 추가
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
# 실제 학습 진행
with tf.Session() as sess: 
  sess.run(tf.global_variables_initializer())
  
  # 10000 Step 최적화 수행
  for i in range(10000):
    batch = next_batch(128, x_train, y_train_one_hot.eval())

    # 100 Step마다 training 데이터셋에 대한 정확도와 loss 출력
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
      loss_print = loss.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})

      print("반복(Epoch): %d, 트레이닝 데이터 정확도: %f, 손실 함수(loss): %f" % (i, train_accuracy, loss_print))
    # 20% 확률의 Dropout을 이용해서 학습 진행
    sess.run(train_step, feed_dict={x: batch[0], y: batch[1], keep_prob: 0.8})

  # 학습이 끝나면 테스트 데이터(10000개)에 대한 정확도를 출력  
  test_accuracy = 0.0  
  for i in range(10):
    test_batch = next_batch(1000, x_test, y_test_one_hot.eval())
    test_accuracy = test_accuracy + accuracy.eval(feed_dict={x: test_batch[0], y: test_batch[1], keep_prob: 1.0})
  test_accuracy = test_accuracy / 10;
  print("테스트 데이터 정확도: %f" % test_accuracy)

반복(Epoch): 0, 트레이닝 데이터 정확도: 0.132812, 손실 함수(loss): 135.744675
반복(Epoch): 100, 트레이닝 데이터 정확도: 0.117188, 손실 함수(loss): 2.515899
반복(Epoch): 200, 트레이닝 데이터 정확도: 0.242188, 손실 함수(loss): 2.069960
반복(Epoch): 300, 트레이닝 데이터 정확도: 0.218750, 손실 함수(loss): 2.169713
반복(Epoch): 400, 트레이닝 데이터 정확도: 0.351562, 손실 함수(loss): 1.883044
반복(Epoch): 500, 트레이닝 데이터 정확도: 0.429688, 손실 함수(loss): 1.565647
반복(Epoch): 600, 트레이닝 데이터 정확도: 0.335938, 손실 함수(loss): 1.854719
반복(Epoch): 700, 트레이닝 데이터 정확도: 0.531250, 손실 함수(loss): 1.491057
반복(Epoch): 800, 트레이닝 데이터 정확도: 0.562500, 손실 함수(loss): 1.461697
반복(Epoch): 900, 트레이닝 데이터 정확도: 0.531250, 손실 함수(loss): 1.252760
반복(Epoch): 1000, 트레이닝 데이터 정확도: 0.562500, 손실 함수(loss): 1.203849
반복(Epoch): 1100, 트레이닝 데이터 정확도: 0.515625, 손실 함수(loss): 1.317173
반복(Epoch): 1200, 트레이닝 데이터 정확도: 0.531250, 손실 함수(loss): 1.459018
반복(Epoch): 1300, 트레이닝 데이터 정확도: 0.554688, 손실 함수(loss): 1.303578
반복(Epoch): 1400, 트레이닝 데이터 정확도: 0.523438, 손실 함수(loss): 1.220593
반복(Epoch): 1500, 트레이닝 데이터 정확도: 0.570312, 손실 함수(loss): 1.115151
반복